In [1]:
import numpy as np
from tqdm import tqdm
from functools import reduce

In [2]:
with open("./data_inputs/day19_input.txt") as f:
    input = f.read().split("\n")

In [3]:
c = 0
scanners = []
for l in input:
    if l[0:3] == "---":
        scanner = []
    elif len(l) == 0:
        scanners.append(np.array(scanner))
    else:
        scanner.append(list(map(int, l.split(","))))



In [4]:
# ----- PART 1 -----

def internal_beacons_pattern(scanner):
    
    pattern = np.array([np.array([np.sqrt(np.sum((scanner_i - scanner_j)**2)) for scanner_j in scanner]) for scanner_i in scanner])  
    
    return pattern

In [5]:
patterns = [internal_beacons_pattern(scanner) for scanner in scanners]      # patterns 

In [6]:
print("scanners len:", len(scanners))
print("scanner0 beacons:", scanners[0].shape)
print("28 x 26 x 3")
print("\npatterns len:", len(patterns))
print("pattern0 shape:", patterns[0].shape, "26 distances for every single beacon of 26 beacons of the scanner 0 ")
print("28 x 26 x 26")

scanners len: 28
scanner0 beacons: (26, 3)
28 x 26 x 3

patterns len: 28
pattern0 shape: (26, 26) 26 distances for every single beacon of 26 beacons of the scanner 0 
28 x 26 x 26


In [7]:
coincidents = {}

for i, p1 in tqdm(enumerate(patterns)):
    for j, p2 in enumerate(patterns):
        if i >= j:
            continue

        coincident = np.zeros((p1.shape[0], p2.shape[0]))

        for k, b1 in enumerate(p1):
            for l, b2 in enumerate(p2):
                
                coincident[k, l] = sum([dist_b1 in b2 for dist_b1 in b1])

        coincidents[(i, j)] = coincident
        


28it [00:25,  1.10it/s]


In [8]:
beacons = []

for key_c, coincident in coincidents.items():
    coincident_b = np.array(np.where(coincident > 3)).T
    if coincident_b.size > 0:
        for beacon in coincident_b:
            beacons.append(((key_c[0], beacon[0]), (key_c[1], beacon[1])))

    

In [9]:
beacons_list_sets = [{beacons[0][0], beacons[0][1]}]

for beacon in beacons:
    found = False
    for beacon_set in beacons_list_sets:
        if beacon[0] in beacon_set:
            found = True
            beacon_set.add(beacon[1])
            break
        elif beacon[1] in beacon_set:
            found = True
            beacon_set.add(beacon[0])
            break
       
    if not found:
        beacons_list_sets.append({beacon[0], beacon[1]})



In [10]:
for i, beacon_set_i in enumerate(beacons_list_sets):
    for beacon in beacon_set_i:
        for j, beacon_set_j in enumerate(beacons_list_sets):
            if i == j:
                continue

            if beacon in beacon_set_j:
                beacon_set_i.update(beacon_set_j)
                beacons_list_sets.pop(j)


In [11]:
all_beacons = [[(i, j) for j, beacon in enumerate(scanner)] for i, scanner in enumerate(scanners)]
all_beacons = reduce(lambda x, y: x + y, all_beacons)

In [12]:
for beacon in all_beacons:
    found = False
    for beacon_set in beacons_list_sets:
        if beacon in beacon_set:
            found = True
            break

    if not found:
        beacons_list_sets.append({beacon})

In [13]:
print("Result1:", len(beacons_list_sets))

Result1: 362


In [14]:
# ----- PART 2 -----

neighbors = {i: [] for i in range(28)}             # having >= 12 shared beacons

for key_c, coincident in coincidents.items():
    coincident_b = np.array(np.where(coincident > 3)).T
    if coincident_b.size > 12:
        neighbors[key_c[0]].append(key_c[1])
        neighbors[key_c[1]].append(key_c[0])  

In [15]:
beacons_list_dict = list(map(dict, beacons_list_sets))

def find_shared_beacons(i, j, scanners):
    shared_beacons = []

    for beacon_dict in beacons_list_dict:
        if (i in beacon_dict.keys()) and (j in beacon_dict.keys()):
            shared_beacons.append(np.array([scanners[i][beacon_dict[i]], scanners[j][beacon_dict[j]]]))

    return np.array(shared_beacons)

In [16]:
sign_array = np.array([[1, 1, 1],
                    [1, 1, -1], 
                    [1, -1, 1],
                    [1, -1, -1],
                    [-1, 1, 1],
                    [-1, 1, -1],
                    [-1, -1, 1],
                    [-1, -1, -1],
                    ])

permutations = np.array([[0, 1, 2], [0, 2, 1], [2, 0, 1], [1, 2, 0], [2, 1, 0], [1, 0, 2]])


def scanners_dist(shared_beacons, permutation1, sign1):              # relative dist from scanner0 to scanner1
    beacons1 = shared_beacons[:, 0, permutation1] * sign1
    #print(beacons1)

    for permutation2 in permutations:
        for sign2 in sign_array:
            beacons2 = shared_beacons[:, 1, permutation2] * sign2
            dif = beacons1 - beacons2
            #print("perm2:", permutation2, "sign2:", sign2, "\ndif:\n", dif)
            if np.all(dif == dif[0]):
                return dif[0], permutation2, sign2
            
    print("error")


In [17]:
locked_scanners = {0: ([0, 1, 2], [1, 1, 1])}
scanners_distances = {}
scanners_coords = {0: np.zeros(3)}

def lock_next_scanner(scanner):
    i = scanner
    neighbors_j = neighbors[i]
    #print("i", i, "\tneighbors j:", neighbors[i])
    #print(scanner_distances)
    
    if len(neighbors_j) > 0:  
        for j in neighbors_j:
            if (i, j) not in scanners_distances:
                shared_beacons = find_shared_beacons(i, j, scanners)
                #print(shared_beacons)
                dist_ij, permutation, sign = scanners_dist(shared_beacons, *locked_scanners[i])

                scanners_distances[(i, j)] = dist_ij
                scanners_distances[(j, i)] = dist_ij * -1
                if j not in scanners_coords:
                    scanners_coords[j] = scanners_coords[i] + dist_ij
                locked_scanners[j] = (permutation, sign)

                lock_next_scanner(j)
        

In [20]:
lock_next_scanner(0)

In [21]:
max_manhattan_dist = 0

for coords1 in scanners_coords.values():
    for coords2 in scanners_coords.values():
        dist = coords1 - coords2
        dist = np.sum(np.abs(dist[0]) + np.abs(dist[1]) + np.abs(dist[2]))
        if dist > max_manhattan_dist:
            max_manhattan_dist = dist

print("Result2:", max_manhattan_dist)


Result2: 12204.0
